In [1]:
%pwd

'/Users/shreeharshmshivpuje/MLpr/TEXTTextSummarization-AWS-DOCKER-GITHUBACTIONS/TextSummarization-AWS-DOCKER-GITHUBACTIONS/research'

In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'/Users/shreeharshmshivpuje/MLpr/TEXTTextSummarization-AWS-DOCKER-GITHUBACTIONS/TextSummarization-AWS-DOCKER-GITHUBACTIONS'

In [4]:
!pip uninstall -y box python-box
!pip install python-box==6.0.2
!pip show python-box


Found existing installation: python-box 6.0.2
Uninstalling python-box-6.0.2:
  Successfully uninstalled python-box-6.0.2
  Using cached python_box-6.0.2-py3-none-any.whl.metadata (7.7 kB)
Using cached python_box-6.0.2-py3-none-any.whl (26 kB)
Name: python-box
Version: 6.0.2
Summary: Advanced Python dictionaries with dot notation access
Home-page: https://github.com/cdgriffith/Box
Author: Chris Griffith
Author-email: chris@cdgriffith.com
License: MIT
Location: /Users/shreeharshmshivpuje/MLpr/TEXTTextSummarization-AWS-DOCKER-GITHUBACTIONS/TextSummarization-AWS-DOCKER-GITHUBACTIONS/venv/lib/python3.11/site-packages
Requires: 
Required-by: 


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from textsummarization_aws_docker_githubactions.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH

from textsummarization_aws_docker_githubactions.utils.common import read_yaml_file, create_directory



2025-09-14 10:18:18,641 - text_summarization_logger - INFO - Logger has been configured. - __init__


In [6]:
import box
print(box.__file__)  # Shows the path to the python-box package
from box import Box  # Should work now


/Users/shreeharshmshivpuje/MLpr/TEXTTextSummarization-AWS-DOCKER-GITHUBACTIONS/TextSummarization-AWS-DOCKER-GITHUBACTIONS/venv/lib/python3.11/site-packages/box/__init__.py


In [7]:
PARAMS_FILE_PATH
CONFIG_FILE_PATH

PosixPath('config/config.yaml')

In [ ]:
class configrationManeger:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml_file(config_filepath)
        # self.params = read_yaml_file(params_filepath)

        create_directory([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directory([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config

In [9]:
import os
from urllib import request
from zipfile import ZipFile
from textsummarization_aws_docker_githubactions.logging import logger
from textsummarization_aws_docker_githubactions.utils.common import get_size_of_file

In [10]:
import tqdm

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self) -> Path:
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size_of_file(Path(self.config.local_data_file))}")
        
        return Path(self.config.local_data_file)
    
    def unzip_and_clean(self, zip_file_path: Path) -> None:
        logger.info(f"Unzipping file: {zip_file_path} into dir: {self.config.unzip_dir}")
        with ZipFile(zip_file_path, 'r') as zip_ref:
            for file in tqdm.tqdm(iterable=zip_ref.namelist(), total=len(zip_ref.namelist())):
                zip_ref.extract(member=file, path=self.config.unzip_dir)
        logger.info(f"Unzipping completed!")
        
        # removing zip file
        os.remove(zip_file_path)
        logger.info(f"Removed zip file: {zip_file_path} after extraction")
        
    def initiate_data_ingestion(self) -> Path:
        zip_file_path = self.download_file()
        self.unzip_and_clean(zip_file_path=zip_file_path)
        
        return self.config.unzip_dir

In [20]:
params = read_yaml_file(params_filepath)
print("Params content:", params, type(params))


NameError: name 'params_filepath' is not defined

In [13]:
try:
    config = configrationManeger()
    data_ingestion_config = config.get_data_ingestion_config()
    
    print(data_ingestion_config)
    
    data_ingestion = DataIngestion(config=data_ingestion_config)
    unzip_dir = data_ingestion.initiate_data_ingestion()
    
    print(f"Data ingested at: {unzip_dir}")
except Exception as e:
    raise e

BoxKeyError: "'Box' object has no attribute 'root_dir'"